In [5]:
from langgraph.graph import StateGraph
from pydantic import BaseModel
import os

# Define State for Code Generation Workflow
class CodeGenState(BaseModel):
    requirements: str
    generated_project_path: str | None = None

# Worker to create project folder
def create_folder_structure(state: CodeGenState) -> CodeGenState:
    """Creates project folder structure."""
    project_name = "GeneratedApp"
    base_path = os.path.join(os.getcwd(), project_name)
    os.makedirs(base_path, exist_ok=True)
    
    state.generated_project_path = base_path
    return state

# Worker to generate API code
def generate_api_code(state: CodeGenState) -> CodeGenState:
    """Generates API code and writes to a file."""
    file_path = os.path.join(state.generated_project_path, "api.py")
    with open(file_path, "w") as f:
        f.write("# Auto-generated API code")

    return state

# Define Orchestrator Workflow
orchestrator_graph = StateGraph(CodeGenState)
orchestrator_graph.add_node("create_folder_structure", create_folder_structure)
orchestrator_graph.add_node("generate_api_code", generate_api_code)
orchestrator_graph.set_entry_point("create_folder_structure")
orchestrator_graph.add_edge("create_folder_structure", "generate_api_code")

# Compile the Orchestrator Workflow
orchestrator_workflow = orchestrator_graph.compile()
orchestrator_workflow


from langgraph.graph import StateGraph
from pydantic import BaseModel


# Define State for Basic Workflow
class MainState(BaseModel):
    user_authenticated: bool = False
    requirements: str | None = None
    generated_project_path: str | None = None

# Worker to authenticate user
def authenticate_user(state: MainState) -> MainState:
    """Simulates user authentication."""
    state.user_authenticated = True
    return state

# Worker to call the Orchestrator Workflow
def run_orchestrator(state: MainState) -> MainState:
    """Calls the Orchestrator Workflow."""
    print("in")
    if not state.requirements:
        raise ValueError("Requirements not provided!")

    # Run Orchestrator-Worker Workflow
    orchestrator_state = orchestrator_workflow.invoke(
        {"requirements": state.requirements}
    )

    # Store the generated project path in the main workflow
    state.generated_project_path = orchestrator_state["generated_project_path"]
    return state

# Define the Main Workflow Graph
main_graph = StateGraph(MainState)
main_graph.add_node("authenticate_user", authenticate_user)
main_graph.add_node("run_orchestrator", run_orchestrator)

# Define Workflow Order
main_graph.set_entry_point("authenticate_user")
main_graph.add_edge("authenticate_user", "run_orchestrator")

# Compile the Main Workflow
main_workflow = main_graph.compile()




def main():
    print("Starting Main Workflow...")

    # Provide application requirements
    state = {
        "requirements": "E-commerce app with login, product catalog, and checkout."
    }

    # Run the workflow
    final_state = main_workflow.invoke(state)

    print("Workflow completed!")
    print(f"Generated project path: {final_state['generated_project_path']}")

if __name__ == "__main__":
    main()




Starting Main Workflow...
in
Workflow completed!
Generated project path: e:\AgenticAI\Hackathon\GeneratedApp
